# import

In [1]:
import os
import pdb
import time
from pprint import pprint

import pandas as pd
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm

from utils import RecruiterAgent

In [2]:
import requests
import json

In [3]:
# Model
load_dotenv(find_dotenv()) # read local .env file
assert os.environ["JIGSAW_API_KEY"]  != ''
assert os.environ["JIGSAW_PUBLIC_KEY"]  != ''
assert os.environ["GROQ_API_KEY"]  != ''

# Jigsaw

In [4]:
JIGSAW_API_KEY = os.environ["JIGSAW_API_KEY"]

In [5]:
JIGSAW_PUBLIC_KEY = os.environ["JIGSAW_PUBLIC_KEY"]

## upload

In [6]:
# Define the endpoint and the API key
endpoint = "https://api.jigsawstack.com/v1/store/file?key=gettysburg.wav&overwrite=true"
print(endpoint)

# Define the headers
headers = {
    "Content-Type": "audio/wav",
    "x-api-key": JIGSAW_API_KEY,
}

https://api.jigsawstack.com/v1/store/file?key=gettysburg.wav&overwrite=true


In [7]:
%%time
# Define the body of the request
body = open('data/gettysburg.wav', 'rb')

# Make the POST request
response = requests.post(endpoint,
                         headers=headers,
                         data=body
                        )

# Parse the JSON response
data = response.json()

# Print the result
print(data)

{'success': True, 'url': 'https://api.jigsawstack.com/v1/store/file/gettysburg.wav', 'key': 'gettysburg.wav'}
CPU times: user 20.5 ms, sys: 0 ns, total: 20.5 ms
Wall time: 7.68 s


## transcribe

In [8]:
# Define the endpoint and the API key
endpoint = "https://api.jigsawstack.com/v1/ai/transcribe"

# Define the headers
headers = {
    "Content-Type": "application/json",
    "x-api-key": JIGSAW_API_KEY,
}

# Define the body of the request
body = {
    "file_store_key": "gettysburg.wav",
}

In [9]:
print(endpoint)
print(headers)
print(body)

https://api.jigsawstack.com/v1/ai/transcribe
{'Content-Type': 'application/json', 'x-api-key': 'sk_acd188330c1f9f68e2bc78bd43d8166965a8dad216eb01cb95a0a338b962ef30639891eb20133044ef621920a9f2dde49c00b5f0eedb6977219e1843f0d7688d024AFgZoUHfu6FCv1Hjg4'}
{'file_store_key': 'gettysburg.wav'}


In [10]:
%%time
# Make the POST request
response = requests.post(endpoint,
                         headers=headers,
                         data=json.dumps(body)
                        )

# Parse the JSON response
data = response.json()

# Print the result
print(data)

{'success': True, 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure', 'chunks': [{'timestamp': [0, 17.58], 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure'}]}
CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 35.8 s


In [11]:
%%time
# Define the endpoint and the API key
endpoint = "https://api.jigsawstack.com/v1/ai/transcribe"

# Define the headers
headers = {
    "Content-Type": "application/json",
    "x-api-key": JIGSAW_API_KEY,
}

# Define the body of the request
body = {
    "file_store_key": "gettysburg.wav",
}

# Make the POST request
response = requests.post(endpoint,
                         headers=headers,
                         data=json.dumps(body)
                        )

# Parse the JSON response
data = response.json()

# Print the result
print(data)

{'success': True, 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure', 'chunks': [{'timestamp': [0, 17.58], 'text': ' four score and seven years ago our fathers brought forth on this continent a new nation conceived in liberty and dedicated to the proposition that all men are created equal now we are engaged in a great civil war testing whether that nation or any nation so conceived and so dedicated can long endure'}]}
CPU times: user 7.54 ms, sys: 6.47 ms, total: 14 ms
Wall time: 4.51 s
